In [7]:
class Device(object):
    """Each  of the factury line:
    Assumtion:
        We are not sending the values that are zero or negative when creating the class(we filter them out)
    Attributes:
        id: A string representing the Device id  .
        errors: dictinary of the errors,   id as the key  and their MTTF and MeanTimeDown as values [ErrorType,MTTF,MeanTimeDown]
        Line number: the current line 
        Run Time: the run time of the device  , dictianry with error id as the key , MTTF- running time of the device since the last maintance for that error as value if it gets close to a threshold we notify the system admion 
    """
    
    def __init__(self, deviceId,Line):
        "Initilize the values"
        self.id = deviceId
        self.line=Line
        self.errors={}#dict()
        self.runTime={}
        self.maintenanceFlag=False  # this should be array for each error but for now just use one flag 
        
    def get_deviceId(self):
        return self.id
    def get_line(self):
        return self.line
    def get_errors(self):
        return self.errors
    def get_runTime(self):
        return self.runTime
    
    def update_deviceId(self, newDeviceId):
        self.id=newDeviceId
    def update_line(self,newLine):
        self.line=newLine
    def get_error_size(self):
        '''Return the number of errors that are associated with each device '''
        return len(self.errors)
    
    def add_errors(self,deviceId,errorID,MTTF,MeanDownT,ErrorType):
        "Assign the values of error associated with the device"
        if(self.id!=deviceId):
            print("WRONG DEVICE!")
            return
        if(errorID not in self.errors):
            self.errors[errorID]=[ErrorType,MTTF,MeanDownT]
        #else --Not sure about this yet 
        #we also initilize the values in the runTime dictinary
        #if(errorID not in self.runTime and ErrorType=='Unplanned'):
        if(errorID not in self.runTime): 
            self.runTime[errorID]=MTTF
        
    def update_RunTime(self, hours):
        "updating the run time after some amopunt of running time of the device"
        for errorID in self.runTime:
            self.runTime[errorID]-=hours
            
    def reset_RunTime(self, errorID):
        "if a fix for an error id happens for a device the runtime value will be updated to the original value "
        if((errorID  in self.runTime) and (errorID in self.errors)):
            self.maintenanceFlag=False  # if it was the array then we use the error to index it 
            self.runTime[errorID]=self.runTime[errors][0]
               
    def get_maintenanceFlag(self):
        return self.maintenanceFlag
            
    def perdicted_error(self, threshold):
        "Return a list of errors for the device that can happen in the next given threshold time"
        maintenance=[]
        for errorID in self.runTime:
            if self.runTime[errorID] <= threshold:
                    self.maintenanceFlag=True
                    maintenance.append(errorID)
        return maintenance
    
    def print_device(self):
        print (self.runTime)
            
    

In [6]:
'''Connecting to DB'''
import pyodbc
Server='qa-datagov01.nexjqa.local'
DB='RPO_MAJIK_DISCOVERY'
Ver='sql2012'
user='nexj'
password='nexj'

cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+Server+';DATABASE='+DB+';UID='+user+';PWD='+password)
# Create a cursor from the connection
cursor = cnxn.cursor()

Error: ('01000', "[01000] [unixODBC][Driver Manager]Can't open lib 'SQL Server' : file not found (0) (SQLDriverConnect)")

In [ ]:
sqlStr='SELECT [id]  FROM [RPO_MAJIK_DISCOVERY].[dbo].[equipment]'  
cursor.execute(sqlStr)    
buffers = cursor.fetchall()
equipments=[]
# eqStr="("

for buffer in buffers: 
    equipments.append (buffer[0]) 
#print (equipments)

In [4]:
eqi=[]
for equipment in equipments:
        
        sqlStr=' select * FROM [RPO_MAJIK_DISCOVERY].[dbo].[MTTFtbl]  WHERE LTRIM(RTRIM([DeviceId]))=LTRIM(RTRIM(\''+equipment+'\')) AND  [MTTF] > 0 ' 
#         print (sqlStr)
        cursor.execute(sqlStr)    
        '''Read all at once '''
        rows = cursor.fetchall()
        buffer=Device(equipment,1)
        for row in rows:
            buffer.add_errors(equipment,row[1],row[4],row[5],row[2])
        eqi.append(buffer)
# print(eqi)

NameError: name 'equipments' is not defined

In [4]:
for e in eqi:
    print((e.get_deviceId(),e.get_error_size()))

NameError: name 'eqi' is not defined

In [8]:
for e in eqi:
    # update the time for errors (let's say that we have been running it for 5 hours)
    e.update_RunTime(5)
    #put the threashold to 1 hour 
    needToFix=e.perdicted_error(1)
    #if one of the errors break then we print them and fix them

    if(e.get_maintenanceFlag()==True):
        print(needToFix)
        print ("FIXING")
        for error in needToFix:
            e.reset_RunTime(e)
        
    

NameError: name 'eqi' is not defined